#online

In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [95]:
# We open the main file to analyse what the data are and how they are sorted
bc_df = pd.read_csv('MAIN_EN_Berlin_bike_thief_file_for_analysis.csv')

In [96]:
lor_df = pd.read_csv('Berlin_LOR_code.csv', sep=';')
#lor_df2 = pd.read_csv('Berlin_LOR_code2_enc.csv')

In [97]:
# to avoid any issue by reading the german special character we encode it to 'latin-1' to change them, and then re-encode it 'utf-8'
bcUpd_df = pd.read_csv('Fahrraddiebstahl_Final.csv', sep=';')
bcUpd_df

,ANGELEGT_AM,TATZEIT_ANFANG_DATUM,TATZEIT_ANFANG_STUNDE,TATZEIT_ENDE_DATUM,TATZEIT_ENDE_STUNDE,LOR,SCHADENSHOEHE,VERSUCH,ART_DES_FAHRRADS,DELIKT,ERFASSUNGSGRUND
0,05.02.2023,05.02.2023,8,05.02.2023,8,11200514,0,Nein,Mountainbike,Keller- und Bodeneinbruch,Sonstiger schwerer Diebstahl in/aus Keller/Bod...
1,05.02.2023,04.02.2023,10,04.02.2023,11,1300730,372,Nein,Fahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von FahrrÃ¤dern
2,05.02.2023,05.02.2023,6,05.02.2023,9,6300632,749,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von FahrrÃ¤dern
3,05.02.2023,04.02.2023,12,05.02.2023,8,12100205,350,Nein,Herrenfahrrad,Keller- und Bodeneinbruch,Sonstiger schwerer Diebstahl in/aus Keller/Bod...
4,05.02.2023,04.02.2023,16,05.02.2023,13,2500728,1500,Nein,Damenfahrrad,Keller- und Bodeneinbruch,Sonstiger schwerer Diebstahl in/aus Keller/Bod...
...,...,...,...,...,...,...,...,...,...,...,...
2716,01.12.2022,29.11.2022,18,30.11.2022,7,8100312,1005,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von FahrrÃ¤dern
2717,01.12.2022,30.11.2022,9,30.11.2022,11,2200208,1002,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von FahrrÃ¤dern
2718,01.12.2022,30.11.2022,9,30.11.2022,9,2400521,3000,Nein,Fahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von FahrrÃ¤dern
2719,01.12.2022,28.11.2022,19,28.11.2022,22,7300619,700,Nein,Herrenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von FahrrÃ¤dern


# Standardizing

We start by unifying the date in the european format

In [98]:
bc_df['Record_date'] = pd.to_datetime(bc_df['Record_date'])
bc_df['Stealing_date'] = pd.to_datetime(bc_df['Stealing_date'])
bc_df['Report_stealing_date'] = pd.to_datetime(bc_df['Report_stealing_date'])

We now unify the columns of "Fahrraddiebstahl_U.csv" which is the updated records to be then concatenate to the main file "bc_df".

By looking at the columns of both files we will adapt to the main file column which is bc_df.

Green column : original data provided by the berliner police department<br />
Blue column : addition for date precision<br />


In [99]:
bcUpd_df['ANGELEGT_AM'] = pd.to_datetime(bcUpd_df['ANGELEGT_AM'])
bcUpd_df['TATZEIT_ANFANG_DATUM'] = pd.to_datetime(bcUpd_df['TATZEIT_ANFANG_DATUM'])
bcUpd_df['TATZEIT_ENDE_DATUM'] = pd.to_datetime(bcUpd_df['TATZEIT_ENDE_DATUM'])

/Users/david/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '31.01.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/david/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '30.01.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/david/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '29.01.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/david/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '28.01.2023

In [21]:
#We drop the date below december 2022 for avoiding duplicate
bcUpd_df = bcUpd_df[bcUpd_df['ANGELEGT_AM'] > '2022-12-01']

Creating a separate file to implemente the updates

In [100]:
bc_df_final = bc_df[['Record_date','Record_day', 'Record_month', 'Record_year',
       'Stealing_date', 'Stealing_hour', 'Report_stealing_date',
       'Report_stealing_hour', 'Berlin_code_area', 'Bike_value',
       'Unsuccesful_attempt', 'Bike_type', 'Crime_type', 'Crime_reason',
       'Administrative_district', 'District', 'Quarter', 'Street_name']].copy()

**Translation**<br />
We rename and translate the column and their value in english 

In [102]:
bcUpd_df.columns

Index(['ANGELEGT_AM', 'TATZEIT_ANFANG_DATUM', 'TATZEIT_ANFANG_STUNDE',
       'TATZEIT_ENDE_DATUM', 'TATZEIT_ENDE_STUNDE', 'LOR', 'SCHADENSHOEHE',
       'VERSUCH', 'ART_DES_FAHRRADS', 'DELIKT', 'ERFASSUNGSGRUND'],
      dtype='object')

In [103]:
bcUpd_df.rename(columns={'ANGELEGT_AM': 'Record_date', 'TATZEIT_ANFANG_DATUM': 'Report_stealing_date', 'TATZEIT_ANFANG_STUNDE': 'Report_stealing_hour', 'TATZEIT_ENDE_DATUM': 'Stealing_date', 'TATZEIT_ENDE_STUNDE': 'Stealing_hour', 'LOR': 'Berlin_code_area', 'SCHADENSHOEHE': 'Bike_value', 'VERSUCH': 'Unsuccesful_attempt', 'ART_DES_FAHRRADS': 'Bike_type', 'DELIKT': 'Crime_type', 'ERFASSUNGSGRUND': 'Crime_reason'}, inplace=True)

In [104]:
bcUpd_df['Unsuccesful_attempt'] = bcUpd_df['Unsuccesful_attempt'].replace('Nein', "No")
bcUpd_df['Unsuccesful_attempt'] = bcUpd_df['Unsuccesful_attempt'].replace('Ja', "Yes")
bcUpd_df['Unsuccesful_attempt'] = bcUpd_df['Unsuccesful_attempt'].replace('Unbekannt', "Unknown")

In [105]:
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Mountainbike', "Mountain")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Fahrrad', "Bike")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Herrenfahrrad', "Men")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Damenfahrrad', "Women")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Rennrad', "Racing")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('diverse Fahrräder', "Multiple bikes")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Kinderfahrrad', "Child")
bcUpd_df['Bike_type'] = bcUpd_df['Bike_type'].replace('Lastenfahrrad', "Cargo")

In [106]:
bcUpd_df['Crime_type'] = bcUpd_df['Crime_type'].replace('Keller- und Bodeneinbruch', "Break-in")
bcUpd_df['Crime_type'] = bcUpd_df['Crime_type'].replace('Fahrraddiebstahl', "Theft")

In [107]:
bcUpd_df['Crime_type'] = bcUpd_df['Crime_type'].replace('Keller- und Bodeneinbruch', "Break-in")
bcUpd_df['Crime_type'] = bcUpd_df['Crime_type'].replace('Fahrraddiebstahl', "Theft")

In [108]:
bc_df_final['Crime_reason'].unique()

array(['Aggravated theft', 'Aggravated theft from basement/floor',
       'Theft', 'Theft from basement/floor'], dtype=object)

In [109]:
bcUpd_df['Crime_reason'].unique()

array(['Sonstiger schwerer Diebstahl in/aus Keller/Boden von FahrrÃ¤dern',
       'Sonstiger schwerer Diebstahl von FahrrÃ¤dern',
       'Einfacher Diebstahl von FahrrÃ¤dern',
       'Einfacher Diebstahl aus Keller/Boden von FahrrÃ¤dern'],
      dtype=object)

In [110]:
bcUpd_df['Crime_reason'] = bcUpd_df['Crime_reason'].replace('Sonstiger schwerer Diebstahl in/aus Keller/Boden von FahrrÃ¤dern', "Aggravated theft from basement/floor")
bcUpd_df['Crime_reason'] = bcUpd_df['Crime_reason'].replace('Sonstiger schwerer Diebstahl von FahrrÃ¤dern', "Aggravated theft")
bcUpd_df['Crime_reason'] = bcUpd_df['Crime_reason'].replace('Einfacher Diebstahl von FahrrÃ¤dern', "Theft")
bcUpd_df['Crime_reason'] = bcUpd_df['Crime_reason'].replace('Einfacher Diebstahl aus Keller/Boden von FahrrÃ¤dern', "Theft from basement/floor")

We now adapt the date format of Record_date from bcUpd_df to fit with the bc_df_final one.
And we precise the dates and time<br />
- Record_day/Record_month/Record_year<br />
- Stealing_month	Stealing_year

We enriche the dates and time by the months and years to add more futur filtering possibilities :

In [111]:
bcUpd_df["stealing_month"] = pd.DatetimeIndex(bcUpd_df['Stealing_date']).month
bcUpd_df["stealing_year"] = pd.DatetimeIndex(bcUpd_df['Stealing_date']).year
bcUpd_df["stealing_day"] = pd.DatetimeIndex(bcUpd_df['Stealing_date']).day

In [112]:
#using function pd.DatetimeIndex to extract the month and the year from a date.
bc_df_final["stealing_month"] = pd.DatetimeIndex(bc_df_final['Stealing_date']).month
bc_df_final["stealing_year"] = pd.DatetimeIndex(bc_df_final['Stealing_date']).year
bc_df_final["Stealing_day"] = pd.DatetimeIndex(bc_df_final['Stealing_date']).day

**MERGING**

The data now fits in terms of name and date. We can concatenate both files :

In [113]:
bc_merge = bc_df_final.merge(lor_df['Berlin_code_area'], left_index=True, right_index=True, how='left')

In [114]:
#bc_merge = bc_merge.drop('Berlin_code_area_y', axis=1)
bc_merge.rename(columns={'Berlin_code_area_x': 'Berlin_code_area'}, inplace=True)

In [115]:
BC_FINAL = pd.concat([bc_merge,bcUpd_df], axis=0)

In [116]:
BC_FINAL = BC_FINAL.merge(lor_df['Berlin_code_area'], left_index=True, right_index=True, how='left')

In [117]:
lor_df.head()

,Berlin_code_area,Administrative_district,District,Quarter,Street_name
0,1100101,Mitte,Zentrum,Tiergarten Süd,Stülerstraße
1,1100102,Mitte,Zentrum,Tiergarten Süd,Großer Tiergarten
2,1100103,Mitte,Zentrum,Tiergarten Süd,Lützowstraße
3,1100104,Mitte,Zentrum,Tiergarten Süd,Körnerstraße
4,1100205,Mitte,Zentrum,Regierungsviertel,Wilhelmstraße


In [118]:
bcUpd_df = bcUpd_df.merge(lor_df['Berlin_code_area'], left_index=True, right_index=True, how='right')

In [119]:
bcUpd_df.to_csv('sepa.csv', index=False)

In [120]:
BC_FINAL

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area_x,Bike_value,...,Administrative_district,District,Quarter,Street_name,stealing_month,stealing_year,Stealing_day,Berlin_code_area_y,stealing_day,Berlin_code_area_y
0,2022-12-01,1.0,12.0,2022.0,2022-12-01,8,2022-12-01,8,1300836,506,...,Mitte,Gesundbrunnen,Brunnenstraße Nord,Humboldthain Nordwest,12,2022,1.0,1100101.0,NaN,1100101.0
0,2023-05-02,NaN,NaN,NaN,2023-05-02,8,2023-05-02,8,11200514,0,...,NaN,NaN,NaN,NaN,5,2023,NaN,NaN,2.0,1100101.0
1,2022-12-01,1.0,12.0,2022.0,2022-11-29,22,2022-11-30,17,1300732,718,...,Mitte,Gesundbrunnen,Osloer Straße,Soldiner Straße,11,2022,29.0,1100102.0,NaN,1100102.0
1,2023-05-02,NaN,NaN,NaN,2023-04-02,11,2023-04-02,10,1300730,372,...,NaN,NaN,NaN,NaN,4,2023,NaN,NaN,2.0,1100102.0
2,2022-12-01,1.0,12.0,2022.0,2022-05-28,7,2022-05-28,14,6300526,569,...,Steglitz-Zehlendorf,Zehlendorf Süd/Lichterfelde West,Teltower Damm,Zehlendorf Mitte,5,2022,28.0,1100103.0,NaN,1100103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40689,2021-01-02,2.0,1.0,2021.0,2021-01-02,13,2021-01-02,13,8401243,1700,...,Neukölln,Buckow Nord/Rudow,Rudow,Alt-Rudow,1,2021,2.0,NaN,NaN,NaN
40690,2021-01-01,1.0,1.0,2021.0,2021-01-01,5,2021-01-01,8,10400941,2527,...,Marzahn-Hellersdorf,Kaulsdorf/Mahlsdorf,Mahlsdorf,Pilgramer Straße,1,2021,1.0,NaN,NaN,NaN
40691,2021-01-01,1.0,1.0,2021.0,2021-01-01,17,2021-01-01,18,8401241,888,...,Neukölln,Buckow Nord/Rudow,Rudow,Blumenviertel,1,2021,1.0,NaN,NaN,NaN
40692,2021-01-01,1.0,1.0,2021.0,2021-01-01,9,2021-01-01,11,4200206,290,...,Charlottenburg-Wilmersdorf,Charlottenburg West,Heerstraße,Kranzallee,1,2021,1.0,NaN,NaN,NaN


In [121]:
BC_FINAL.to_csv('LAST ONE.csv', index=False)